In [13]:
# Imports
import pandas as pd

# Dependencies (webapp)
from ipywidgets import interact
import ipywidgets as widgets
from io import BytesIO

import matplotlib.pyplot as plt
import plotly.express as px

### Upload transactions file 
Make sure the file has "amount" and "currency" columns.

In [8]:
# Initialize outputs
# Upload file and Upload button
upload_btn_output = widgets.Output(clear_output=True)
display(upload_btn_output)

data_process_output = widgets.Output(clear_output=True)
display(data_process_output)

Output()

Output()

### Filtered Data

In [9]:
#Filter Toggle Output
filter_toggle_output = widgets.Output(clear_output=True)
display(filter_toggle_output)

#Print out filtered
data_output = widgets.Output(clear_output=True)
display(data_output)

Output()

Output()

### Bar Chart

In [10]:
chart_output = widgets.Output()
display(chart_output)

Output()

In [5]:
#Event handler for upload button
global data

def upload_btn_eventhandler(obj):

    #Read file
    for file_name in upload_file.value:
        
        #Split file name to extract data
        extension = file_name.split('.')[1]
        content = upload_file.value[file_name]['content']

        if(extension == 'xlsx' or extension == 'xls'):
            data = pd.read_excel(BytesIO(content))
            
        elif(extension == 'csv'):
            data = pd.read_csv(BytesIO(content))
        
        else:
            print("File not accepted")
            
        # Get indexes where value is > 1000
        data = data[data.amount < 1000]

        with data_process_output:
            print(f"Processed a total of {len(data)} transactions.")
            
    filter_data(data)
    plot_data(data)
    
    return data

# Buttons
# Upload
upload_file = widgets.FileUpload(multiple=False)
upload_btn = widgets.Button(description="Upload", icon='check', button_style='success')
upload_btn.on_click(upload_btn_eventhandler)

input_widgets = widgets.HBox([upload_file, upload_btn])
with upload_btn_output:
    display(input_widgets)

In [6]:
# Filter - Helper
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique = [str(x) for x in unique]
    unique.sort()
    unique.insert(0, ALL)
    return unique

# Filtering Pt-1
def filter_data(data):
    
    # Creating Dropdowns and buttons
    dropdown_currency = widgets.Dropdown(options=data.currency.unique(), description='Currency:')

    # Event handler for dropdowns 
    def dropdown_currency_eventhandler(change):
        common_filtering(data, change.new)
    
    #Call
    dropdown_currency.observe(dropdown_currency_eventhandler, names='value')

    #Layout box
    input_widgets = widgets.HBox([dropdown_currency])
    
    with filter_toggle_output:
        display(input_widgets)

# Filtering Pt-2
def common_filtering(data, currency):
    
    common_filter = data[(data.currency == currency)]
        
    with data_output:
        display(common_filter)
    
    return common_filter

# Bar chart
def plot_data(data):
    
    data['Count'] = 1
    plot_data = data[['amount','currency','Count']]
    plot_data = plot_data.groupby(["currency"],as_index=False).Count.sum()
    
    with chart_output:
        print("Total data: ", len(data))
        plot_bar(plot_data, x="currency", y="Count", color="currency")

# Bar Chart helper
def plot_bar(data, x, y, color):

    fig = px.bar(data, x, y, color, orientation='v',width=1000)
    
    fig.update_layout(
    plot_bgcolor='white', # clean plot
    )
    
    return fig.show()